In [1]:
import sys
# do this only once
sys.path.append('./src/audio_processing/')
sys.path.append('./src/data_processing/')
sys.path.append('./src/chord_parser/')
sys.path.append('./src/metrics/')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#python input/output and regex
import re
import os
from pathlib import Path

#chord info
import pychord as pc

#signal processing libraries
from scipy.io import wavfile
from scipy import signal

#sklearn for normalization
from sklearn.preprocessing import StandardScaler, OneHotEncoder

#tensorflow
import tensorflow as tf
from tensorflow import keras

#librosa
import librosa
import librosa.display

#import custom modules
import filters
import spectrograms
import audiofiles
import chord_parser
import annotation_processing
import metrics

## Data Augmentation
There are classes of chords that doesn't appear very ofter, so in order to balance our dataset, we will have to replicate the existing data in other pitches

In [ ]:
nbins=192
bins_per_octave=24
hop_length=2048
sampling_rate= 22050

audiofiles_path = 'Audiofiles/The Beatles/original'
shifted_audiofiles_path = 'Audiofiles/The Beatles/'

In [5]:
for filename in Path(audiofiles_path).glob('**/wav/*.wav'):

    path, track = os.path.split(filename)
    path, wav = os.path.split(path)
    path, album = os.path.split(path)

    track_no = re.search('([0-9].).', track).group(1)

    y, sr = librosa.load(filename, sr=sampling_rate)

    y_shifted = librosa.effects.pitch_shift(y, sr, n_steps=-3, bins_per_octave=24)
    
    librosa.output.write_wav(shifted_audiofiles_path + 'down3' + '/' + album + '/' + track_no + '_down3.wav', y_shifted, sr=sampling_rate, norm=False)

In [ ]:
for filename in Path(audiofiles_path).glob('**/wav/*.wav'):

    path, track = os.path.split(filename)
    path, wav = os.path.split(path)
    path, album = os.path.split(path)

    track_no = re.search('([0-9].).', track).group(1)

    y, sr = librosa.load(filename, sr=sampling_rate)

    y_shifted = librosa.effects.pitch_shift(y, sr, n_steps=-2, bins_per_octave=24)
    
    librosa.output.write_wav(shifted_audiofiles_path + '/' + album + '/' + track_no + '_down2.wav', y_shifted, sr=sampling_rate, norm=False)

In [ ]:
for filename in Path(audiofiles_path).glob('**/wav/*.wav'):

    path, track = os.path.split(filename)
    path, wav = os.path.split(path)
    path, album = os.path.split(path)

    track_no = re.search('([0-9].).', track).group(1)

    y, sr = librosa.load(filename, sr=sampling_rate)

    y_shifted = librosa.effects.pitch_shift(y, sr, n_steps=-1, bins_per_octave=24)
    
    librosa.output.write_wav(shifted_audiofiles_path + '/' + album + '/' + track_no + '_down1.wav', y_shifted, sr=sampling_rate, norm=False)

In [ ]:
for filename in Path(audiofiles_path).glob('**/wav/*.wav'):

    path, track = os.path.split(filename)
    path, wav = os.path.split(path)
    path, album = os.path.split(path)

    track_no = re.search('([0-9].).', track).group(1)

    y, sr = librosa.load(filename, sr=sampling_rate)

    y_shifted = librosa.effects.pitch_shift(y, sr, n_steps=1, bins_per_octave=24)
    
    librosa.output.write_wav(shifted_audiofiles_path + '/' + album + '/' + track_no + '_up1.wav', y_shifted, sr=sampling_rate, norm=False)

In [ ]:
for filename in Path(audiofiles_path).glob('**/wav/*.wav'):

    path, track = os.path.split(filename)
    path, wav = os.path.split(path)
    path, album = os.path.split(path)

    track_no = re.search('([0-9].).', track).group(1)

    y, sr = librosa.load(filename, sr=sampling_rate)

    y_shifted = librosa.effects.pitch_shift(y, sr, n_steps=2, bins_per_octave=24)
    
    librosa.output.write_wav(shifted_audiofiles_path + '/' + album + '/' + track_no + '_up2.wav', y_shifted, sr=sampling_rate, norm=False)

In [6]:
for filename in Path(audiofiles_path).glob('**/wav/*.wav'):

    path, track = os.path.split(filename)
    path, wav = os.path.split(path)
    path, album = os.path.split(path)

    track_no = re.search('([0-9].).', track).group(1)

    y, sr = librosa.load(filename, sr=sampling_rate)

    y_shifted = librosa.effects.pitch_shift(y, sr, n_steps=3, bins_per_octave=24)
    
    librosa.output.write_wav(shifted_audiofiles_path + 'up3' + '/' + album + '/' + track_no + '_up3.wav', y_shifted, sr=sampling_rate, norm=False)

### Transcriptions

In [7]:
target = 'Big-Dataset/The Beatles/scratch/'

In [29]:
ChordLib = {'C' : 0, 'C#': 1, 'Db' : 1, 'D' : 2, 'D#' : 3, 'Eb' : 3, 'E' : 4, 'Fb' : 4, 'F' : 5, 'F#' : 6, 'Gb' : 6, 'G' : 7, 'G#' : 8, 'Ab' : 8, 'A' : 9, 'A#' : 10, 'Bb' : 10, 'B' : 11, 'Cb' : 11, 'N' : 12, 'X' : 13}
semitone_list = ['C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B']

In [38]:
### get all chordlab files
Chordlab = {'The Beatles': {}}
for filename in Path('Big-Dataset/The Beatles').glob('**/scratch/original/*/*.lab'):
    
    path, track = os.path.split(filename)
    path, album = os.path.split(path)
    track_no = re.search('([0-9].)_-_',track).group(1)
    
    if (album not in Chordlab['The Beatles']): 
        Chordlab['The Beatles'][album] = {}
        
    Chordlab['The Beatles'][album][track_no] = pd.read_csv(filename, names=['Starts', 'Ends', 'Chord'], sep=' ', header=None)
    
    # down 3
    chord_list = []
    for _, starts, ends, chord in Chordlab['The Beatles'][album][track_no].itertuples():
        rest = ''
        if chord != 'N':
            if chord.find(':') != -1:
                chord, rest = chord.split(':')
                rest = ':' + rest
            if chord.find('/') != -1:
                chord, rest = chord.split('/')
                rest = '/' + rest
            chord_list.append([starts, ends, semitone_list[ChordLib[chord] - 3] + rest])
        else:
            chord_list.append([starts, ends, chord])
    
    df = pd.DataFrame(chord_list, columns = ['Starts', 'Ends', 'Chord'])
    newfilename = target + 'down3' + '/' + album + '_down3/' + track
    df.to_csv(newfilename, sep = ' ', index=False, header=False)
    
    # up 3
    chord_list = []
    for _, starts, ends, chord in Chordlab['The Beatles'][album][track_no].itertuples():
        rest = ''
        if chord != 'N':
            if chord.find(':') != -1:
                chord, rest = chord.split(':')
                rest = ':' + rest
            if chord.find('/') != -1:
                chord, rest = chord.split('/')
                rest = '/' + rest
            chord_list.append([starts, ends, semitone_list[(ChordLib[chord] + 3)%12] + rest])
        else:
            chord_list.append([starts, ends, chord])
    
    df = pd.DataFrame(chord_list, columns = ['Starts', 'Ends', 'Chord'])
    newfilename = target + 'up3' + '/' + album + '_up3/' + track
    df.to_csv(newfilename, sep = ' ', index=False, header=False)